# computation of shape metrics: parallelization

In [24]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math, random, time, os, sys, string
import geopandas as gpd
from shapely.geometry import Polygon, Point, MultiPolygon, LineString

In [25]:
shp_path = '/Users/rodrigo/Documents/tfg/data/ups/ups_3395F.gpkg'
shpfile = gpd.read_file(shp_path)
pf = shpfile['geometry'][0]

In [4]:
"python setup.py build_ext --inplace"

'python setup.py build_ext --inplace'

In [26]:
import shapemetrics

def shapemetrics_pol(polygon):
    
    featPntLst = shapemetrics.ConvertToGridPnts(polygon)
    centroid = polygon.centroid
    Xc, Yc = centroid.x, centroid.y
    A = polygon.area
    r = (A / math.pi)**.5       # radius of equal area circle (circle with area equal to shape area)

    # COHESION #
    interdist = shapemetrics.interpointDistance(featPntLst)
    circ_interD = r * .9054
    CohesionIndex = circ_interD / interdist
    
    # PROXIMITY #
    D_to_Center, EAC_pix = shapemetrics.proximity(featPntLst,Xc,Yc,r)    
    # avg distance to center for equal area circle...
    circD = r * (2.0/3.0)
    # proximity index (circle / shape)...
    ProximityIndex = circD / D_to_Center         

    # SPIN #
    shpMOI = shapemetrics.spin(featPntLst,Xc,Yc)
    # moment of inertia for equal area circle...
    circ_MOI = .5 * r**2
    # calculate spin index (circle / shape)...
    Spin = circ_MOI / shpMOI    
    
    return A, r, CohesionIndex, ProximityIndex, Spin

In [30]:
from joblib import Parallel, delayed
num_cores = 6
metrics = Parallel(n_jobs = num_cores, verbose = 1)(delayed(shapemetrics_pol)(pol) for pol in shpfile['geometry'][0:8])

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 out of   8 | elapsed:    2.4s remaining:    0.8s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:    4.0s finished


In [32]:
metricsdf = pd.DataFrame(metrics, columns=['A', 'r','Cohesion', 'proximity', 'spin'])
metricsdf

,A,r,Cohesion,proximity,spin
0,3.094636e+07,3138.555710,0.905991,0.913345,0.814558
1,3.793492e+07,3474.918906,0.922441,0.924614,0.835310
2,3.493896e+07,3334.878695,0.925010,0.931242,0.847344
3,3.693504e+07,3428.817289,0.931248,0.937375,0.858605
4,3.893138e+07,3520.261931,0.928868,0.934673,0.856355
5,3.893138e+07,3520.261931,0.927116,0.930280,0.848284
6,3.992995e+07,3565.122428,0.923491,0.927796,0.844064
7,4.592056e+07,3823.214537,0.921178,0.924916,0.839148


In [ ]:

shpfilemetrics.to_file('/Users/rodrigo/Documents/tfg/data/ups/shapemetrics.shp')
# I need to do it for the actual shapes AND for the potential shapes!
# Cythonize spin and proximity

In [29]:
metrics

,A,r,Cohesion,proximity,spin
0,3.094636e+07,3138.555710,0.910156,0.913345,0.814558
1,3.793492e+07,3474.918906,0.917168,0.924614,0.835310
2,3.493896e+07,3334.878695,0.924389,0.931242,0.847344
3,3.693504e+07,3428.817289,0.932248,0.937375,0.858605
4,3.893138e+07,3520.261931,0.926758,0.934673,0.856355
5,3.893138e+07,3520.261931,0.926448,0.930280,0.848284
6,3.992995e+07,3565.122428,0.926258,0.927796,0.844064
7,4.592056e+07,3823.214537,0.921739,0.924916,0.839148


In [22]:
pf.area/1000000

30.94635879057193

In [23]:
shpfile.crs

{'init': 'epsg:3395'}

In [ ]:
10**6